In [1]:
import iisignature
import pandas as pd
import numpy as np

from datetime import datetime

from Backtesting.Create_dataframes import CreateDataframe

from Brokers.Alpaca.Alpaca_keyes import API_KEY, SECRET_KEY
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, LSTM

2025-02-05 21:46:32.509554: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Normalize data and instead of predictiong five numbers, just predict one over the 5 days

In [2]:
bank_account = 10000
commission_fee = 0.01
slippage_cost = 0.1

create_df = CreateDataframe()

start_date_train = datetime(2023, 1, 1) 
end_date_train = datetime(2025, 1, 1)

data = create_df.Alpaca(
    api_key=API_KEY,
    secret_key=SECRET_KEY,
    start_date=start_date_train,
    end_date=end_date_train,
    interval=TimeFrame(amount = 1, unit = TimeFrameUnit.Hour),
    asset_list="TSLA"
)

In [3]:
data['date'] = data['timestamp'].dt.date  # Extract the date part
unique_days = {day: i for i, day in enumerate(sorted(data['date'].unique()))}
data['day'] = data['date'].map(unique_days)

In [4]:
data

,timestamp,open,high,low,close,volume,date,day
0,2023-01-03 09:00:00+00:00,120.0200,121.23,115.94,118.5200,1306706.0,2023-01-03,0
1,2023-01-03 10:00:00+00:00,118.5300,120.50,118.20,120.3800,568635.0,2023-01-03,0
2,2023-01-03 11:00:00+00:00,120.2800,120.50,117.95,118.6100,547403.0,2023-01-03,0
3,2023-01-03 12:00:00+00:00,118.6300,118.99,116.61,118.2800,1529445.0,2023-01-03,0
4,2023-01-03 13:00:00+00:00,118.3200,121.90,116.61,118.9300,2886437.0,2023-01-03,0
...,...,...,...,...,...,...,...,...
7971,2024-12-31 20:00:00+00:00,408.5725,409.97,402.54,403.6400,11331536.0,2024-12-31,527
7972,2024-12-31 21:00:00+00:00,403.6400,404.99,401.70,403.3501,5461661.0,2024-12-31,527
7973,2024-12-31 22:00:00+00:00,403.4100,403.60,401.20,401.6400,1148641.0,2024-12-31,527
7974,2024-12-31 23:00:00+00:00,401.6000,402.66,401.60,402.5000,172880.0,2024-12-31,527


In [5]:
output_data = data.groupby("day").agg(
    close=("close", "last")
)

output_data["return"] = output_data["close"].pct_change() #periods=n I tried to predict 5 days, what if we just predict 1?
output_data["state"] = np.where(output_data["return"] >= 0, 1, 0)

In [6]:
output_data

,close,return,state
day,,,
0,107.2700,NaN,0
1,113.5400,0.058451,1
2,110.1500,-0.029857,0
3,113.0600,0.026419,1
4,120.3000,0.064037,1
...,...,...,...
524,430.9200,-0.049686,0
525,431.9404,0.002368,1
526,413.5000,-0.042692,0


In [7]:
def leadlag(X, use_time = False):
   if (not use_time):
       lead = []
       lag = []
       for val_lag, val_lead in zip(X[:-1], X[1:]):
           lead.append(val_lag)
           lag.append(val_lag)
           lead.append(val_lead)
           lag.append(val_lag)
       lead.append(X[-1])
       lag.append(X[-1])
       return np.c_[lead, lag]
   else:
       lead = []
       lag = []
       t = []
       time = 0
       t.append(time)
       lag.append(X[0])
       lead.append(X[0])
       for val_lag, val_lead in zip(X[:-1], X[1:]):
           time += 1
           t.append(time)
           lead.append(val_lag)
           lag.append(val_lag)
           t.append(time)
           lead.append(val_lead)
           lag.append(val_lag)
           t.append(time)
           lead.append(val_lead)
           lag.append(val_lead)
       return np.c_[lead, lag, t]

In [8]:
# Prepare input
X_list = []

n_days = len(output_data)
for i in range(n_days - 30 - 5):
    window = data[(data["day"] >= i)&(data["day"] < i + 30)]
    path = window["close"].to_numpy()
    ll_path = leadlag(path)
    x = iisignature.sig(ll_path, 13)
    X_list.append(x.reshape(1, -1))

X = np.vstack(X_list)

In [9]:
y_list = []

for i in range(31, n_days-4):
    window = output_data[(output_data.index >= i)&(output_data.index < i + 5)]
    res = window["state"].to_numpy()[-1] # .reshape(1, -1)
    y_list.append(res)

y = np.vstack(y_list)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [11]:
mu = np.mean(X_train)
sigma2 = np.std(X_train)

X_train = (X_train - mu) / sigma2
X_test = (X_test - mu) / sigma2

In [12]:
model = Sequential()
model.add(Dense(32, input_dim=X_train.shape[1], activation="relu"))
model.add(Dense(32, activation="relu"))
model.add(Dense(1, activation="tanh"))

/Users/doblerloic/Desktop/Finance_prediction_project/predictive_finance/.venv/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
print(X_train.shape[1])

In [13]:
model.compile(optimizer="adam", loss="mse", metrics=["accuracy"])
model.fit(X_train, y_train, epochs=60, batch_size=32)

Epoch 1/60
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.4716 - loss: 0.5073
Epoch 2/60
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5251 - loss: 0.3420
Epoch 3/60
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5151 - loss: 0.2860
Epoch 4/60
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5443 - loss: 0.2604
Epoch 5/60
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5195 - loss: 0.2680 
Epoch 6/60
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4908 - loss: 0.2504 
Epoch 7/60
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5201 - loss: 0.2543 
Epoch 8/60
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5457 - loss: 0.2465 
Epoch 9/60
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5244 - loss: 0.2511  
Epoch 10/60
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4983 - loss: 0.2576 
Epoch 11/60
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4888 - loss: 0.2660
Epoch 12/60
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 

In [14]:
pred = model.predict(X_test)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step


In [15]:
y_test

array([[1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
    

In [16]:
print(pred)

[[ 0.85752267]
 [ 0.48411492]
 [ 0.4839588 ]
 [ 0.48388895]
 [ 0.48256102]
 [ 0.483723  ]
 [ 0.48772502]
 [ 0.48394454]
 [ 0.4812401 ]
 [ 0.51959497]
 [ 0.5862237 ]
 [ 0.4841008 ]
 [ 0.48336118]
 [ 0.4935935 ]
 [ 0.60958654]
 [ 0.48393422]
 [ 0.54923356]
 [ 0.48381606]
 [ 0.48054332]
 [-1.        ]
 [ 0.48251688]
 [ 0.48393285]
 [ 1.        ]
 [ 0.48433506]
 [ 0.4839232 ]
 [ 0.6223264 ]
 [ 0.8871837 ]
 [ 1.        ]
 [ 0.48382434]
 [ 0.48393708]
 [ 0.49724957]
 [ 0.4839852 ]
 [ 0.48459044]
 [ 0.5384219 ]
 [ 0.48026672]
 [ 0.46802264]
 [ 0.48433506]
 [ 0.48546985]
 [-0.05377646]
 [ 0.48395044]
 [ 0.48751044]
 [ 0.49098358]
 [ 0.4839995 ]
 [ 0.48397315]
 [ 0.43089634]
 [ 0.5440914 ]
 [ 1.        ]
 [ 0.480053  ]
 [ 0.47858202]
 [ 0.48391443]
 [ 0.48280475]
 [ 0.48424256]
 [-0.08657148]
 [ 0.48144275]
 [ 0.4846296 ]
 [-0.04307798]
 [ 0.49168736]
 [-0.99682146]
 [ 0.48386484]
 [ 0.48399258]
 [ 0.3811137 ]
 [ 0.48350957]
 [ 0.5812109 ]
 [ 0.48383954]
 [ 0.99998456]
 [ 0.48377156]
 [ 0.48397

In [17]:
print(np.abs(pred - y_test) < 0.5)

[[ True]
 [False]
 [False]
 [False]
 [ True]
 [False]
 [ True]
 [ True]
 [ True]
 [False]
 [False]
 [False]
 [ True]
 [False]
 [False]
 [ True]
 [ True]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [ True]
 [False]
 [False]
 [ True]
 [False]
 [ True]
 [False]
 [False]
 [False]
 [False]
 [ True]
 [ True]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [ True]
 [ True]
 [False]
 [ True]
 [ True]
 [ True]
 [False]
 [ True]
 [False]
 [ True]
 [False]
 [ True]
 [ True]
 [False]
 [False]
 [False]
 [False]
 [ True]
 [False]
 [ True]
 [ True]
 [ True]
 [False]
 [ True]
 [ True]
 [ True]
 [False]
 [ True]
 [False]
 [False]
 [ True]
 [False]
 [False]
 [ True]
 [False]
 [ True]
 [ True]
 [False]
 [False]
 [False]
 [ True]
 [False]
 [ True]
 [ True]
 [ True]
 [False]
 [ True]
 [ True]
 [ True]
 [ True]
 [False]
 [False]
 [False]
 [ True]
 [False]
 [ True]
 [ True]
 [False]
 [False]
 [ True]
 [False]
 [False]
 [False]
 [ True]
 [ True]
 [ True]
 [ True]
 [False]
 [False]
 [ True]
 

In [18]:
clf = Ridge(alpha=1)
clf.fit(X_train, y_train)

Ridge(alpha=1)

In [19]:
rid = clf.predict(X_test)

In [21]:
rid

array([[ 0.80469182],
       [ 0.50287798],
       [ 0.50293009],
       [ 0.50519196],
       [ 0.50285529],
       [ 0.5029065 ],
       [ 0.50733262],
       [ 0.50290965],
       [ 0.50182281],
       [ 0.50985501],
       [ 0.53419924],
       [ 0.5005432 ],
       [ 0.50288564],
       [ 0.5026221 ],
       [ 0.52529608],
       [ 0.50291594],
       [ 0.52704386],
       [ 0.50271683],
       [ 0.50074415],
       [-8.81120364],
       [ 0.5029168 ],
       [ 0.50291827],
       [11.82705249],
       [ 0.50288676],
       [ 0.50290863],
       [ 0.52110821],
       [ 0.94888507],
       [ 1.43235658],
       [ 0.50287607],
       [ 0.50290277],
       [ 0.51327611],
       [ 0.502919  ],
       [ 0.50229623],
       [ 0.50297379],
       [ 0.50292157],
       [ 0.50175601],
       [ 0.50299011],
       [ 0.50001746],
       [ 0.20293592],
       [ 0.50293009],
       [ 0.50382984],
       [ 0.50659398],
       [ 0.502929  ],
       [ 0.50294121],
       [ 0.50405218],
       [ 0

In [20]:
print(np.abs(rid-y_test) > 0.51)

[[False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [ True]
 [False]
 [False]
 [False]
 [ True]
 [False]
 [False]
 [False]
 [False]
 [ True]
 [False]
 [False]
 [ True]
 [False]
 [False]
 [ True]
 [False]
 [ True]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [ True]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [ True]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [ True]
 [False]
 [ True]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [ True]
 [False]
 [False]
 [False]
 [False]
 [False]
 [ True]
 [False]
 [ True]
 [False]
 [ True]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [ True]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [ True]
 [False]
 [False]
 [ True]
 [False]
 [False]
 [False]
 

In [ ]:
y_test